In [1]:
import torch

In [5]:
# Adjust the path as necessary to point to your specific split file
split_data_path = '/Users/nash/Project/fedn/fedn/examples/mnist-pytorch/data/clients/5/dataset.pt'
split_data = torch.load(split_data_path)


In [6]:
split_data.keys()

dict_keys(['input_ids', 'labels'])

In [7]:
len(split_data['input_ids'])

2492